## Modelos de series temporales (datos de tráfico de la M30 de Madrid)

In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt

## Descripción del caso

En este caso vamos a realizar un análisis sencillo de series temporales. En este ejemplo no realizaremos un modelado complejo ni predicciones a futuro, pero utilizaremos las mismas métricas que se usan para valorar esas predicciones para comparar distintos casos de indicadores de tráfico.

In [4]:
trafico_raw = pd.read_csv(r'C:\Users\andre\Downloads\traficoNov2019\traficoM30.csv', encoding = 'latin1', sep = ';', parse_dates = ['fecha'])
trafico_raw.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2019-11-01 00:00:00,M30,1320,3.0,0,50.0,N,5
1,1001,2019-11-01 00:15:00,M30,1260,3.0,0,54.0,N,5
2,1001,2019-11-01 00:30:00,M30,948,3.0,0,56.0,N,5
3,1001,2019-11-01 00:45:00,M30,888,3.0,0,59.0,N,5
4,1001,2019-11-01 01:00:00,M30,972,3.0,0,59.0,N,5


In [5]:
# Fitramos los datos de interés para este análisis
trafico = trafico_raw[trafico_raw['tipo_elem'] == 'M30'][['id', 'fecha', 'intensidad', 'vmed' ]]
trafico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1179781 entries, 0 to 10655110
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          1179781 non-null  int64         
 1   fecha       1179781 non-null  datetime64[ns]
 2   intensidad  1179781 non-null  int64         
 3   vmed        1175182 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 45.0 MB


In [10]:
trafico_raw['id'].value_counts()
#Vemos que hay id's que cubren la serie temporal completa (2880) mientras otros
# que no. La serie temporal de interes requiere 2880 data points que es lo 
# equivalente a 30 dias de recoleccion de data con intervalos de 15 minutos.
#Este es justamente el intervalo con el que se están presentando cada uno de 
# los id's. 2880 * 15 = 43200 minutos --> 30 dias.

# A efectos de este caso, nos sirve eliminar la mayoria de las filas,
# y quedarnos con aquellos que cumplen la serie temporal de 2880
# ya que hay datos más que suficientes para el analisis

id
1001    2880
5975    2880
3916    2880
5968    2880
5967    2880
        ... 
6048       1
6347       1
4689       1
5867       1
5869       1
Name: count, Length: 4039, dtype: int64

In [51]:
# Algunos de los puntos de medida no tienen la serie temporal completa; a efectos de este caso, nos 
# sirve con eliminar esos casos, ya que hay datos más que suficientes para el análisis
datos_completos = trafico.id.value_counts() == 2880
ids_datos_completos = datos_completos[datos_completos == True].index.to_list()

In [16]:
datos_completos = trafico['id'].value_counts() == 2880
ids_datos_completos = datos_completos[datos_completos == True].index.to_list()

trafico = trafico[trafico.id.isin(ids_datos_completos)]
trafico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354240 entries, 0 to 9644940
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id          354240 non-null  int64         
 1   fecha       354240 non-null  datetime64[ns]
 2   intensidad  354240 non-null  int64         
 3   vmed        354240 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 13.5 MB


In [17]:
trafico.head()

,id,fecha,intensidad,vmed
0,1001,2019-11-01 00:00:00,1320,50.0
1,1001,2019-11-01 00:15:00,1260,54.0
2,1001,2019-11-01 00:30:00,948,56.0
3,1001,2019-11-01 00:45:00,888,59.0
4,1001,2019-11-01 01:00:00,972,59.0


## Visualización y análisis

En el caso de conjuntos de datos de un cierto tamaño, como es este, el uso de visualizaciones se hace prácticamente imprescindible para poder entender el contenido de los datos.

In [28]:
# Cabe destacar que Intensidad se refiere al flujo de tráfico que pasan por un punto específico. 
# Es una medida de lo congestionado que está el tráfico en esa ubicación.
# Vmed representa la velocidad promedio de los vehículos que pasan por el mismo punto en la carretera durante el 
# período de tiempo especificado.
# Id son los diferentes puntos de medida al rededor de la m30 que podemos tener

In [42]:
data = trafico[trafico.id == 6890]
#Grafico de Intensidad
alt.Chart(data, width=750, height=130)\
    .mark_line()\
    .encode(x='fecha:T', y='intensidad:Q')\
    .interactive(bind_y=False)

alt.Chart(...)

In [45]:
#Grafico de Velocidad Media
alt.Chart(data, width=750, height=130)\
.mark_line()\
.encode(x='fecha:T', y='vmed:Q')\
.interactive(bind_y=False)

alt.Chart(...)

In [49]:
#Haremos los mismos graficos para otro punto de medida, esta vez el 6843.
data2 = trafico[trafico['id'] == 6843]

#Grafico de Intensidad
alt.Chart(data2, width=750, height=130)\
.mark_line()\
.encode(x='fecha:T', y='intensidad:Q')\
.interactive(bind_y = False)

alt.Chart(...)

In [50]:
#Grafico de Velocidad Media
alt.Chart(data2, width=750, height=130)\
.mark_line()\
.encode(x='fecha:T', y='vmed:Q')\
.interactive(bind_y = False)

alt.Chart(...)

Pueden apreciarse patrones temporales claros: existe un componente horario, que hace que por la noches la intensidad sea mucho menor, y se aprecia también una diferencia entre distintos días de la semana.

## Métricas

Vamos a tratar de probar una hipótesis: que el patrón horario de la congestión del tráfico es distinto en diferentes días de la semana. Para ello, haremos la media sobre todos los puntos de control del tráfico y para cada día de la semana, y compararemos los resultados.

In [53]:
trafico.head()

,id,fecha,intensidad,vmed
0,1001,2019-11-01 00:00:00,1320,50.0
1,1001,2019-11-01 00:15:00,1260,54.0
2,1001,2019-11-01 00:30:00,948,56.0
3,1001,2019-11-01 00:45:00,888,59.0
4,1001,2019-11-01 01:00:00,972,59.0


In [111]:
resumen = trafico.groupby([trafico.fecha.dt.weekday, trafico.fecha.dt.time])[['intensidad', 'vmed']].mean().rename_axis(['Dia', 'Hora']).reset_index()
weekdays = {0: 'Lunes', 1: 'Martes', 2: 'Miercoles', 3: 'Jueves', 4: 'Viernes', 5:'Sabado', 6:'Domingo'}
resumen['Dia'] = resumen['Dia'].map(weekdays)
resumen['Hora'] = resumen['Hora'].astype(str)
resumen

,Dia,Hora,intensidad,vmed
0,Lunes,00:00:00,513.991870,55.867886
1,Lunes,00:15:00,435.796748,55.373984
2,Lunes,00:30:00,380.016260,54.178862
3,Lunes,00:45:00,330.138211,51.735772
4,Lunes,01:00:00,279.617886,50.345528
...,...,...,...,...
667,Domingo,22:45:00,853.260163,57.109756
668,Domingo,23:00:00,772.284553,57.650407
669,Domingo,23:15:00,667.308943,57.473577
670,Domingo,23:30:00,628.065041,56.928862


In [112]:
alt.Chart(resumen, width=1200, height=200)\
.mark_line()\
.encode(x= 'Hora:O', y = 'intensidad:Q', color ='Dia:N')
#Podemos observar claramente como en los dias de semana la intensidad es muy baja entre 00:00-06:00 que seria lo esperado
# Ya a partir de las 06:45 podemos ver un gran incremento en la intensidad menos para los dias sabado y domingo.
#Finalmente a horas de la noche (19:30) la intensidad empieza a disminuir significativamente.

alt.Chart(...)

In [113]:
alt.Chart(resumen, width=1200, height=200)\
.mark_line()\
.encode(x='Hora:O', y='vmed:Q', color='Dia:N')
#Vemos que la velocidad media disminuye mucho en la madrugada, y algo en la hora donde mas intensidad hay 07:00-08:00. 

alt.Chart(...)

In [116]:
#Diferencia de estimacion entre Lunes y Martes para la intensidad
mean_squared_error(resumen[resumen.Dia == 'Lunes'].intensidad.to_numpy(), resumen[resumen.Dia == 'Martes'].intensidad.to_numpy(), squared = False)
#Vemos que no hay tanta discrepancia entre la prediccion que se hace de intensidad
# entre el dia lunes y martes, no difieren mucho. Ahora veremos el otro extremo

54.146123493799934

In [117]:
#Diferencia de estimacion entre Lunes y Domingo para la intensidad
mean_squared_error(resumen[resumen.Dia == 'Lunes'].intensidad.to_numpy(), resumen[resumen.Dia == 'Domingo'].intensidad.to_numpy(), squared = False)
#Vemos que la diferencia es mucho mayor que va concorde a las visualizaciones
#generadas anteriormente

642.5349748823916

In [118]:
# Representamos el valor de la diferencia entre días mediante un heatmap

alt.Chart(diffs_RMSE, width=300, height=300).mark_rect()\
        .encode(x = 'Dia de referencia:O', y = 'Dia de comparación:O', color = 'RMSE:Q')

alt.Chart(...)